In [ ]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import numpy as np
import datetime


In [ ]:
# Load dataset
file_path = 'Updated_Indian_Purchase_Dataset.csv'
data = pd.read_csv(file_path)

# Parse purchase dates
data['Purchase Date'] = pd.to_datetime(data['Purchase Date'], dayfirst=True)

# Add a seasonal feature based on the current date
current_month = datetime.datetime.now().month
data['Current Month'] = current_month

# Feature engineering: encode categorical variables
le_gender = LabelEncoder()
data['Gender'] = le_gender.fit_transform(data['Gender'])

le_platform = LabelEncoder()
data['Social Media Platform'] = le_platform.fit_transform(data['Social Media Platform'])

le_timing = LabelEncoder()
data['Social Media Use Timings'] = le_timing.fit_transform(data['Social Media Use Timings'])

le_purchase_platform = LabelEncoder()
data['Purchase Platform'] = le_purchase_platform.fit_transform(data['Purchase Platform'])


In [ ]:
# Features: demographic and product data
features = data[['Gender', 'Age', 'Current Month'] + list(data.columns[9:38])]

# Targets: advertising insights (platform, timing, purchase platform, location)
targets = data[['Social Media Platform', 'Social Media Use Timings', 'Purchase Platform', 'City', 'State']]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import LabelEncoder

for column in y_train.columns:
    le = LabelEncoder()
    y_train[column] = le.fit_transform(y_train[column])




In [ ]:
print(y_test.iloc[:, i].dtype)
print(y_pred_df[column].dtype)



object
int64


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Initialize LabelEncoder
le = LabelEncoder()

# Iterate through target columns and generate classification reports
for i, column in enumerate(targets.columns):
    print(f"Classification Report for {column}:")

    # Encode y_test to numeric labels
    y_test_encoded = le.fit_transform(y_test.iloc[:, i])

    # Ensure y_pred_df[column] is encoded as well (if needed)
    if not np.issubdtype(y_pred_df[column].dtype, np.number):
        y_pred_encoded = le.transform(y_pred_df[column])
    else:
        y_pred_encoded = y_pred_df[column]  # If already numeric

    # Print classification report
    print(classification_report(y_test_encoded, y_pred_encoded, zero_division=1))


Classification Report for Social Media Platform:
              precision    recall  f1-score   support

           0       0.18      0.21      0.19       320
           1       0.18      0.20      0.19       320
           2       0.12      0.10      0.11       358
           3       0.18      0.18      0.18       329
           4       0.15      0.14      0.14       319
           5       0.17      0.15      0.16       354

    accuracy                           0.16      2000
   macro avg       0.16      0.16      0.16      2000
weighted avg       0.16      0.16      0.16      2000

Classification Report for Social Media Use Timings:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         7
           5       0.00      0.

In [ ]:
# Train a multi-output Random Forest model
model = MultiOutputClassifier(RandomForestClassifier(random_state=42))
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=targets.columns)

# Evaluate predictions
for i, column in enumerate(targets.columns):
    print(f"Classification Report for {column}:")




Classification Report for Social Media Platform:
Classification Report for Social Media Use Timings:
Classification Report for Purchase Platform:
Classification Report for City:
Classification Report for State:


In [ ]:
# Encode 'City' and 'State' columns
le_city = LabelEncoder()
data['City'] = le_city.fit_transform(data['City'])

le_state = LabelEncoder()
data['State'] = le_state.fit_transform(data['State'])


In [ ]:
# Dictionary to map month names to numbers
month_dict = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

def generate_recommendation(product, gender, age, month=None):
    # If month is given as a word, convert it to a number
    if isinstance(month, str):
        month = month_dict.get(month.capitalize(), None)

    # If no month is provided or invalid month name, default to current month
    if month is None:
        month = datetime.datetime.now().month

    # Prepare the input data
    input_data = pd.DataFrame({
        'Gender': [le_gender.transform([gender])[0]],
        'Age': [age],
        'Current Month': [month],  # Use the provided or current month
        **{col: [1 if col == product else 0] for col in features.columns[3:]}
    })

    # Predict
    prediction = model.predict(input_data)
    return {
        'Social Media Platform': le_platform.inverse_transform([prediction[0, 0]])[0],
        'Timing': le_timing.inverse_transform([prediction[0, 1]])[0],
        'Purchase Platform': le_purchase_platform.inverse_transform([prediction[0, 2]])[0],
        'State': le_state.inverse_transform([prediction[0, 4]])[0]   # Decode State
    }

# Example usage
# Method 1
print(generate_recommendation('Woolen Clothes', 'Male', 21, month='July'))
print(generate_recommendation('Groceries', 'Male', 71, month='December'))
print(generate_recommendation('Furniture', 'Female', 65, month='January'))
print(generate_recommendation('Laptops', 'Female', 18, month='June'))
print(generate_recommendation('TV', 'Male', 46, month='October'))
print(generate_recommendation('Electronics', 'Female', 19, month='February'))
print(generate_recommendation('Furniture', 'Male', 52, month='March'))
print(generate_recommendation('Shoes', 'Male', 72, month='June'))
print(generate_recommendation('Kitchenware', 'Female', 14, month='August'))
print(generate_recommendation('Kitchenware', 'Male', 35, month='September'))


# Method 2
print(generate_recommendation('Woolen Clothes', 'Male', 25))


{'Social Media Platform': 'Twitter', 'Timing': '15:00 - 21:00', 'Purchase Platform': 'Myntra', 'State': 'West Bengal'}
{'Social Media Platform': 'LinkedIn', 'Timing': '12:00 - 15:00', 'Purchase Platform': 'Myntra', 'State': 'West Bengal'}
{'Social Media Platform': 'Twitter', 'Timing': '7:00 - 9:00', 'Purchase Platform': 'Amazon', 'State': 'Maharashtra'}
{'Social Media Platform': 'Twitter', 'Timing': '8:00 - 12:00', 'Purchase Platform': 'Local Market', 'State': 'Uttar Pradesh'}
{'Social Media Platform': 'Instagram', 'Timing': '7:00 - 21:00', 'Purchase Platform': 'Snapdeal', 'State': 'Uttar Pradesh'}
{'Social Media Platform': 'Twitter', 'Timing': '8:00 - 12:00', 'Purchase Platform': 'Paytm Mall', 'State': 'Uttar Pradesh'}
{'Social Media Platform': 'Instagram', 'Timing': '7:00 - 21:00', 'Purchase Platform': 'Flipkart', 'State': 'Uttar Pradesh'}
{'Social Media Platform': 'LinkedIn', 'Timing': '12:00 - 15:00', 'Purchase Platform': 'Myntra', 'State': 'West Bengal'}
{'Social Media Platform': 

In [ ]:
def generate_recommendation(product, gender, age, current_month):
    # Prepare the input data
    input_data = pd.DataFrame({
        'Gender': [le_gender.transform([gender])[0]],
        'Age': [age],
        'Current Month': [current_month],
        **{col: [1 if col == product else 0] for col in features.columns[3:]}
    })

    # Predict
    prediction = model.predict(input_data)
    return {
        'Social Media Platform': le_platform.inverse_transform([prediction[0, 0]])[0],
        'Timing': le_timing.inverse_transform([prediction[0, 1]])[0],
        'Purchase Platform': le_purchase_platform.inverse_transform([prediction[0, 2]])[0],
        'City': le_city.inverse_transform([prediction[0, 3]])[0],  # Decode City
        'State': le_state.inverse_transform([prediction[0, 4]])[0]   # Decode State
    }

# Example usage
print(generate_recommendation('Groceries', 'Male', 25, current_month))


NameError: name 'current_month' is not defined